In [38]:
import ray
# Disable progress bars and verbose logs
context = ray.data.DataContext.get_current()
context.enable_progress_bars = False
context.verbose = False

2025-06-10 22:00:02,993	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-10 22:00:03,153	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [39]:
ds = ray.data.read_parquet('huggingface_data')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [65]:
ds_without_wit_features = ds.drop_columns(['wit_features'])

In [62]:
print(ds.schema())

Column                           Type
------                           ----
image                            struct<bytes: binary, path: string>
image_url                        string
embedding                        fixed_size_list<item: double>[2048]
metadata_url                     string
original_height                  int32
original_width                   int32
mime_type                        string
caption_attribution_description  string
wit_features                     struct<language: list<item: string>, page_url: list<item: string>, attribution_passes_lang_id: list<item: bool>, caption_alt_text_description: list<item: string>, caption_reference_description: list<item: string>, caption_title_and_reference_description: list<item: string>, context_page_description: list<item: string>, context_section_description: list<item: string>, hierarchical_section_title: list<item: string>, is_main_image: list<item: bool>, page_changed_recently: list<item: bool>, page_title: list<item: 

In [75]:
from typing import Dict
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from PIL import Image
import matplotlib.pyplot as plt
# Pick the largest batch size that can fit on our GPUs.
# If doing CPU inference you might need to lower considerably (e.g. to 10).
BATCH_SIZE = 1024
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
class ImageClassifier:
    def __init__(self):
        self.yolo_model = YOLO(model_path)
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        # Text prompts
        self.prompts = [
            "a photo of a person wearing eyeglasses",
            "a photo of a person wearing sunglasses",
            "a photo of a person not wearing glasses"
        ]
    # BEGIN: Updated __call__ method
    def __call__(self, batch: Dict[str, np.ndarray]):
        ##### Can be made more efficient by using a batch of images instead of processing one by one.
        filtered = []
        truth_values = []
        image_buffer = [np.asarray(Image.open(io.BytesIO(img_dict["bytes"])).convert("RGB")) for img_dict in batch["image"]]
        
        for img_dict in batch["image"]:
            image_buffer = np.asarray(Image.open(io.BytesIO(img_dict["bytes"])).convert("RGB"))
            results = self.yolo_model(image_buffer)
            img_filtered = []
            val = False
            for result in results:
                if (result.boxes.cls.flatten() == 0).any() == False:
                    continue
                x1, y1, x2, y2 = result.boxes.xyxy[0].tolist()
                width = x2 - x1
                height = y2 - y1  
                if width >= 100 and height >= 100:
                    # Get the cropped image and send it to CLIP for classification.
                    cropped_image = image_buffer[int(y1):int(y2), int(x1):int(x2)]
                    pil_image = Image.fromarray(cropped_image)
                    inputs = self.clip_processor(text=self.prompts, images=pil_image, return_tensors="pt", padding=True)
                    outputs = self.clip_model(**inputs)
                    probs = outputs.logits_per_image.softmax(dim=1)
                    max_prob, max_index = probs.max(dim=1)
                    if max_index.item() == 0:  # Check if the highest probability corresponds to the first class
                        val = True
                        img_filtered.append({
                            "x1": x1, "y1": y1,
                            "x2": x2, "y2": y2,
                            "width": width, "height": height,
                            "confidence": max_prob.item()  # Add the confidence value
                        })
            filtered.append(img_filtered)
            truth_values.append(val)
        batch['bounding_boxes'] = filtered
        batch['truth_values'] = truth_values
        return batch     

In [76]:
ds.schema()

Column                           Type
------                           ----
image                            struct<bytes: binary, path: string>
image_url                        string
embedding                        fixed_size_list<item: double>[2048]
metadata_url                     string
original_height                  int32
original_width                   int32
mime_type                        string
caption_attribution_description  string
wit_features                     struct<language: list<item: string>, page_url: list<item: string>, attribution_passes_lang_id: list<item: bool>, caption_alt_text_description: list<item: string>, caption_reference_description: list<item: string>, caption_title_and_reference_description: list<item: string>, context_page_description: list<item: string>, context_section_description: list<item: string>, hierarchical_section_title: list<item: string>, is_main_image: list<item: bool>, page_changed_recently: list<item: bool>, page_title: list<item: 

In [3]:
import pandas as pd
df = pd.read_parquet('huggingface_data/train-00000-of-00330.parquet')

In [77]:
predictions = ds_without_wit_features.map_batches(ImageClassifier,concurrency=1,batch_size=5)

In [79]:
prediction_batch = predictions.take_batch(5)

2025-06-10 22:35:26,309	INFO logging.py:290 -- Registered dataset logger for dataset dataset_25_0
2025-06-10 22:35:26,315	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_25_0. Full logs are in /tmp/ray/session_2025-06-10_22-00-05_944589_88671/logs/ray-data
2025-06-10 22:35:26,315	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_25_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> ActorPoolMapOperator[MapBatches(drop_columns)->MapBatches(ImageClassifier)] -> LimitOperator[limit=5]
2025-06-10 22:35:57,285	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_25_0 execution finished in 30.97 seconds


(_MapWorker pid=37515) Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 0: 480x640 (no detections), 111.7ms
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) Speed: 6.6ms preprocess, 111.7ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 0: 448x640 (no detections), 94.9ms
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) Speed: 2.3ms preprocess, 94.9ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) 0: 448x640 (no detections), 108.5ms
(MapWorker(MapBatches(drop_columns)->MapBatches(ImageClassifier)) pid=37515) Sp

In [32]:
import io
import cv2
yolo_model = YOLO(model_path)
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
prompts = [
            "a photo of a person wearing eyeglasses",
            "a photo of a person wearing sunglasses",
            "a photo of a person not wearing glasses"
        ]
def cv2_to_pil(cv2_img):
    # Convert BGR (OpenCV) to RGB (PIL uses RGB)
    rgb = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)
def check_now(img_dict):
    image_buffer = np.asarray(Image.open(io.BytesIO(img_dict["bytes"])).convert("RGB"))
    results = yolo_model(image_buffer)
    filtered = []
    for result in results:
        if (result.boxes.cls.flatten() == 0).any() == False:
            continue
        x1, y1, x2, y2 = result.boxes.xyxy[0].tolist()
        width = x2 - x1
        height = y2 - y1
        if width >= 100 and height >= 100:
            # Get the cropped image and send it to CLIP for classification.
            cropped_image = image_buffer[int(y1):int(y2), int(x1):int(x2)]
            pil_image = cv2_to_pil(cropped_image)
            inputs = clip_processor(text=prompts, images=pil_image, return_tensors="pt", padding=True)
            outputs = clip_model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
            max_prob, max_index = probs.max(dim=1)
            if max_index.item() == 0:  # Check if the highest probability corresponds to the first class
                filtered.append({
                    "x1": x1, "y1": y1,
                    "x2": x2, "y2": y2,
                    "width": width, "height": height,
                    "confidence": max_prob.item()  # Add the confidence value
                })
    return filtered

In [84]:
images = [
    np.asarray(Image.open(io.BytesIO(img_dict["bytes"])).convert("RGB"))
    for img_dict in df.head()["image"]
]

In [87]:

# Step 1: Convert images from bytes to RGB format
import io
import cv2
yolo_model = YOLO(model_path)
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
prompts = [
            "a photo of a person wearing eyeglasses",
            "a photo of a person wearing sunglasses",
            "a photo of a person not wearing glasses"
        ]

images = [
    np.asarray(Image.open(io.BytesIO(img_dict["bytes"])).convert("RGB"))
    for img_dict in df.head()["image"]
]

# Step 2: Perform YOLO inference on the entire batch
results = yolo_model(images)

# Step 3: Filter bounding boxes based on conditions
filtered_boxes = []
for result in results:
    img_filtered = []
    for box in result.boxes:
        if (box.cls.flatten() == 0).any() == False:
            continue
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        width = x2 - x1
        height = y2 - y1
        if width >= 100 and height >= 100:
            img_filtered.append({
                "x1": x1, "y1": y1,
                "x2": x2, "y2": y2,
                "width": width, "height": height
            })
    filtered_boxes.append(img_filtered)

# Step 4: Crop images based on bounding boxes and prepare for CLIP
cropped_images = []
for img, boxes in zip(images, filtered_boxes):
    for box in boxes:
        x1, y1, x2, y2 = int(box["x1"]), int(box["y1"]), int(box["x2"]), int(box["y2"])
        cropped_images.append(Image.fromarray(img[y1:y2, x1:x2]))

# Step 5: Process cropped images and prompts with CLIP
if cropped_images:
    inputs = clip_processor(
        text=prompts,
        images=cropped_images,
        return_tensors="pt",
        padding=True
    )
    clip_outputs = clip_model(**inputs)
else:
    clip_outputs = None


0: 640x640 (no detections), 85.7ms
1: 640x640 (no detections), 85.7ms
2: 640x640 (no detections), 85.7ms
3: 640x640 (no detections), 85.7ms
4: 640x640 (no detections), 85.7ms
Speed: 3.1ms preprocess, 85.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


In [88]:
clip_m